In [1]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
import csv
import os
from pathlib import Path
from os import listdir
from os.path import isfile, join
import pandas as pd
import re

class LeetcodeScrapper:
    def __init__(self):
        self.driver = Driver(uc=True)
        self.tag_list=[]
        self.companies_list=[]
        self.cno = 0
        self.failed_list =[]
        self.top_question_no_tag_list =[] 
        self.tno = 0
        self.question_dict_list={}
        self.question_queue =[]
        self.qno = 0
        self.qlno =1
        self.find_question_tag_dict_list = {}
        self.find_question_tag_queue = []
        self.qtno = 0
        

    def Login(self,id,pw):
        url = "https://leetcode.com/accounts/login/"
        self.driver.get(url)
        id_field = self.driver.find_element(by=By.ID, value="id_login")
        id_field.send_keys(id)
        time.sleep(1)
        pw_field = self.driver.find_element(by=By.ID, value="id_password")
        pw_field.send_keys(pw)
        #no need check ctp
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.ID, "signin_btn"))).click()
        time.sleep(5)

    def SetTopNo(self,no):
        self.tno = no

    def GetTitleToWebText(self,text):
        text = text.lower()
        text = text.replace('`', '')
        text = text.replace("(", '')
        text = text.replace(")", '')
        text = text.replace(":", '')
        text = text.replace(",", '-')
        text = text.replace("'", '')
        text = text.replace("%", '')
        text = text.replace("&", '')
        text = text.replace(' ', '-')
        text = re.sub(r'(-{2,})', '-', text)
        return text

    def GetQuestionList(self):
        if self.qlno < 29:
            url ="https://leetcode.com/problemset/?sorting=W3sic29ydE9yZGVyIjoiREVTQ0VORElORyIsIm9yZGVyQnkiOiJGUkVRVUVOQ1kifV0%3D&page=" + str(self.qlno)
            self.driver.get(url)
            time.sleep(10)
            with open("Leetcode Overall Question List/"+ str(self.qlno) + ".csv", 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["No", "Title","Link","Tags","Acceptance","Difficulty","Frequency"])
                cols = self.driver.find_elements(by = By.XPATH,value="(//div[@role='cell'])")
                no = str("")
                title = str("")
                link = str("")
                acceptance = str("")
                difficulty = str("")
                i = 1
                for c in cols:
                    column = i % 6
                    if column == 2:
                        text = c.text
                        no = text.split('. ')[0]
                        title = text.split('. ')[1]
                        link = "https://leetcode.com/problems/" + self.GetTitleToWebText(title) +"/description/"
                    elif column == 4 :
                        acceptance = c.text
                    elif column == 5:
                        difficulty = c.text
                    elif column == 0:
                        if no == "2764":
                            writer.writerow(["2764", "is-array-a-preorder-of-some-binary-tree" ,"https://leetcode.com/problems/is-array-a-preorder-of-some-binary-tree/description/","","65.9%","Medium",target_text])
                        else:
                            html_text = c.get_attribute("innerHTML")
                            target_text = html_text.rsplit("style")[1]
                            target_text = target_text.split(";")[0]
                            target_text = target_text.replace('"','')
                            target_text = target_text.replace('=','')
                            target_text = target_text.replace('width','')
                            target_text = target_text.replace(':','')
                            target_text = target_text.replace(' ','')
                            writer.writerow([no, title ,link,"",acceptance,difficulty,target_text])
                    i +=1
                file.close()
            print(self.qlno)
            self.qlno +=1
            self.GetQuestionList()
              

    def GetAllTagAndCompaniesList(self):
        url = "https://leetcode.com/problemset/"
        self.driver.get(url)
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "(//button[@id='headlessui-popover-button-:r5:'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-blue dark:text-dark-blue m-1 flex cursor-pointer items-center px-1'][normalize-space()='Expand'])[1]"))).click()
        tag_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[1]")
        tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('tag.txt', 'w') as f:
            for e in tag_elements:
                self.tag_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-sm leading-5 font-normal text-label-2 dark:text-dark-label-2'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(text(),'Expand')])[1]"))).click()
        companies_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[2]")
        companies_elements = companies_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('companies.txt', 'w') as f:
            for e in companies_elements:
                self.companies_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        time.sleep(2)
        self.GetCompaniesQuestion()

    def TextToWebText(self,text):
        if text == "Quip (Salesforce)":
            return "quip"

        if text == "Virtu Financial":
            return "virtu"
            
        text = text.lower()
        text = text.replace(' ', '-')
        text = text.replace('.', '')
        return text

    def GetCompaniesQuestion(self):
        dest_folder = 'Companies Leetcode'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
        if self.cno < len(self.companies_list):
            company_name = self.companies_list[self.ccno]
            file_name = company_name + ".csv"
            file_path = os.path.join(dest_folder, file_name)
            url = "https://leetcode.com/company/" + self.TextToWebText(company_name) + "/"
            self.driver.get(url)
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//input[@type='checkbox'])[1]"))).click()
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            WebDriverWait(self.driver, 1).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            table = self.driver.find_element(by = By.CSS_SELECTOR, value=".reactable-data")
            rows = table.find_elements(by = By.TAG_NAME, value="tr")
            with open(file_path, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["No", "Title","Link","Tags","Acceptance","Difficulty","Frequency"])

                error =False
                for r in rows:
                    cols = r.find_elements(by = By.TAG_NAME, value="td")
                    if len(cols) == 7:
                        no = cols[1].text
                        title = cols[2].text
                        url = "https://leetcode.com/problems/" + self.GetTitleToWebText(title) +"/description/"
                        tags_list = cols[3].find_elements(by = By.TAG_NAME, value="a")
                        tags = ""
                        for t in tags_list:
                            tags = tags + "[" + t.text + "]"
                        acceptance = cols[4].text
                        difficulty = cols[5].text
                        frequency = cols[6].get_attribute("value")
                        
                        writer.writerow([no, title ,url,tags,acceptance,difficulty,frequency])
                    else:
                        error = True
                file.close()
            if error == True:
                self.failed_list.append(company_name)
                print("Data mining failed.",company_name)
            else:
                print("Done data mining",company_name)
                
            time.sleep(2)
            self.cno += 1
            self.GetCompaniesQuestion()
        else:
            self.driver.quit()
                    
    def InitQuestionDictList(self):
        dir = 'Companies Leetcode'
        filenames = [f for f in listdir(dir) if isfile(join(dir, f))]
        for f in filenames:
            words = f.rsplit(".")
            extension = words[len(words)-1]
            if extension =="csv":
                read_file_path = os.path.join(dir, f)
                df = pd.read_csv(read_file_path)
                for index, row in df.iterrows():
                    no = str(row["No"])
                    title = str(row["Title"])
                    if no not in self.question_dict_list:
                        self.question_dict_list[no] = title
                        self.question_queue.append(no)
        with open("question.csv", 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["No", "Title"])
            for q in self.question_dict_list:
                 writer.writerow([q, self.question_dict_list[q]])
       
            file.close()
            
    def GetQuestion(self):
        if self.qno < len(self.question_queue):
            key = self.question_queue[self.qno]
            title = self.question_dict_list[key]
            path = "question/" + key + ".html"
            url = "https://leetcode.com/problems/" + self.GetTitleToWebText(title) +"/description/"
            self.driver.get(url)
            time.sleep(5)
            with open(path, 'w', encoding='utf-8') as file:
                question_div = self.driver.find_element(by = By.CSS_SELECTOR, value=".elfjS")
                content = question_div.get_attribute("innerHTML")
                file.write(content)  
                file.close()
            print(self.qno,"no",key)  
            self.qno += 1
            time.sleep(1)
            self.GetQuestion()

    def MergeQuestionList(self):
         with open("Question List Without Tag.csv", 'w', newline='',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["No", "Title","Link","Tags","Acceptance","Difficulty","Frequency"])
            for i in range(1, 32):
                path = "Leetcode Overall Question List/" + str(i) + ".csv"
                df = pd.read_csv(path, encoding='latin-1')
                for index, row in df.iterrows():
                    if i == 18:
                        print(index)
                    writer.writerow([str(row['No']), str(row['Title']), str(row['Link']),str(row['Tags']),str(row['Acceptance']),str(row['Difficulty']),str(row['Frequency'])])
             
            file.close()

    def InitFindQuestionTagList(self):
        df = pd.read_csv("Question List - Copy.csv", encoding='latin-1')
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            if tags == "nan":
                link = str(row['Link'])
                if link not in self.find_question_tag_dict_list:
                    self.find_question_tag_dict_list[link] = ""
                    self.find_question_tag_queue.append(link)
        print(len(self.find_question_tag_queue))

    def GetTagForTheQuestion(self, file_path):
        if self.qtno < len(self.find_question_tag_queue):
            tag = ""
            url = self.find_question_tag_queue[self.qtno]
            self.driver.get(url)
            try:
                WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class='flex gap-1'] div:nth-child(2)"))).click()
                tag_elements_parent = self.driver.find_element(by = By.CSS_SELECTOR, value=".mt-2.flex.flex-wrap.gap-1.pl-7")
                time.sleep(1)
                tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="a")
                for e in tag_elements:
                    text = "[" + e.text + "]"
                    tag += text
                file_exists = os.path.exists(file_path)
                if file_exists == False:
                    with open(file_path, 'w', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow(["Link","Tags"])
                        writer = csv.writer(file)
                        writer.writerow([url,tag])
                        file.close()
                else:
                    with open(file_path, 'a', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow([url,tag])
                        file.close()
            except:
                print(url,"no tag")
                file_exists = os.path.exists(file_path)
                if file_exists == False:
                    with open(file_path, 'w', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow(["Link","Tags"])
                        writer = csv.writer(file)
                        writer.writerow([url,""])
                        file.close()
                else:
                    with open(file_path, 'a', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow([url,""])
                        file.close()
            time.sleep(1)
            print(self.qtno) 
            self.qtno += 1
            self.GetTagForTheQuestion(file_path)

    def MergeQuestionTag(self):
        df = pd.read_csv("Question Tag.csv", encoding='latin-1')
        for index, row in df.iterrows():
            link = str(row['Link'])
            if link not in self.find_question_tag_dict_list:
                self.find_question_tag_dict_list[link] = str(row['Tags'])
        with open("Question List.csv", 'w', newline='',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["No", "Title","Link","Tags","Acceptance","Difficulty","Frequency"])
            df = pd.read_csv("Question List - Copy.csv", encoding='latin-1')
            for index, row in df.iterrows():
                link = str(row['Link'])
                tags = str(row['Tags'])
                if link in self.find_question_tag_dict_list:
                    tags = self.find_question_tag_dict_list[link]
                writer.writerow([str(row['No']), str(row['Title']), str(row['Link']),tags,str(row['Acceptance']),str(row['Difficulty']),str(row['Frequency'])])
            file.close()
                
        

In [4]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import time
import csv
import pandas as pd
import multiprocessing 
import threading
import random
import concurrent.futures

class Copliot:
    def __init__(self):
        self.driver = Driver(uc=True)
        self.queue = []
        self.start =0
        self.end = 0
        self.InitQueryList()

    def InitQueryList(self):   
        cf = open("tag.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            self.queue.append(c)
        cf.close
        self.end =len(self.queue)
        print(self.end)
        
    def Login(self,id,pw):
        url = "https://login.live.com/"
        try:
            t = random.uniform(0.0, 2.0)
            print(t,"\n")
            time.sleep(t)
            self.driver.get(url)
            #time.sleep(3)
            #id_field = self.driver.find_element(by=By.XPATH, value="(//input[@id='i0116'])[1]")
            #id_field.send_keys(id)
            #WebDriverWait(self.driver, 1).until(EC.element_to_be_clickable((By.XPATH, "(//button[normalize-space()='Next'])[1]"))).click()
            #time.sleep(2)
            #pw_field = self.driver.find_element(by=By.XPATH, value="(//input[@id='i0118'])[1]")
            #pw_field.send_keys(pw)
            #WebDriverWait(self.driver, 1).until(EC.element_to_be_clickable((By.XPATH, "(//button[normalize-space()='Sign in'])[1]"))).click()
            #time.sleep(2)
        except:
            return False

        return True
        
    def GoToCopliot(self):
        url = "https://www.bing.com/search?q=Bing%20AI&showconv=1&form=MG0AUO"
        # go 3 time to bypass the popup
        self.driver.get(url)
        time.sleep(2)
        self.driver.get(url)
        time.sleep(2)
        return True

    def QueryCopliot(self):
        if self.start < self.end:
            keyword = self.queue[self.start]
            path = "Leetcode Learning Resource/"+keyword+".html"
            url = "https://www.bing.com/search?q=Bing%20AI&showconv=1&form=MG0AUO"
            query = "what is "+ keyword +" in leetcode in one sentence and give me 5 reference links which I can learn for free." 
            dir = path.rsplit("/")[0]
            if not os.path.exists(dir):
                os.makedirs(dir)
            with open(path, 'w', encoding='utf-8') as file:
                self.driver.get(url)
                time.sleep(5)
                dom1 = self.driver.find_element(by=By.CSS_SELECTOR, value=".cib-serp-main").shadow_root
                input_shadow_dom2 = dom1.find_element(by=By.CSS_SELECTOR, value="#cib-action-bar-main").shadow_root
                input_shadow_dom3 = input_shadow_dom2.find_element(by=By.CSS_SELECTOR, value="cib-text-input[serp-slot='none']").shadow_root
                text_field = input_shadow_dom3.find_element(by=By.CSS_SELECTOR, value="#searchbox")
                text_field.send_keys(query)
                time.sleep(1)
                input_shadow_dom2.find_element(by=By.CSS_SELECTOR, value="button[aria-label='Submit']").click()
                time.sleep(30)
                response_shadow_dom2 = dom1.find_element(by=By.CSS_SELECTOR, value="#cib-conversation-main").shadow_root
                response_shadow_dom3 = response_shadow_dom2.find_element(by=By.CSS_SELECTOR, value="cib-chat-turn[serp-slot='none']").shadow_root
                response_shadow_dom4 = response_shadow_dom3.find_element(by=By.CSS_SELECTOR, value=".response-message-group").shadow_root
                response_shadow_dom5 = response_shadow_dom4.find_element(by=By.CSS_SELECTOR, value="cib-message:nth-child(2)").shadow_root
                response = response_shadow_dom5.find_element(by=By.CSS_SELECTOR, value="div[class='content']")
                content = response.get_attribute("innerHTML")
                file.write(content)  
                file.close()
                print(self.start,keyword,"\n")
            file.close()
            self.start += 1
            self.QueryCopliot()
              

    def CloseDriver(self):
        print("run")
        self.driver.Quit()

In [5]:
import pypandoc
from pypandoc.pandoc_download import download_pandoc
import csv
import os
from pathlib import Path
from os import listdir
from os.path import isfile, join
import pandas as pd
import re

class LeetcodeDataProcessing:
    def __init__(self):
        self.companies_list=[]
        self.all_tags_list=[]
        self.InitTagsList()
        self.InitCompaniesList()

    def InitTagsList(self):
        cf = open("tag.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            self.all_tags_list.append(c)
        cf.close

    def GetTitleToWebText(self,text):
        text = text.lower()
        text = text.replace('`', '')
        text = text.replace("(", '')
        text = text.replace(")", '')
        text = text.replace(":", '')
        text = text.replace(",", '-')
        text = text.replace("'", '')
        text = text.replace("%", '')
        text = text.replace("&", '')
        text = text.replace(' ', '-')
        text = re.sub(r'(-{2,})', '-', text)
        return text

    
    def InitCompaniesList(self):
        cf = open("companies.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            self.companies_list.append(c)
        cf.close

    def GenerateLeetCodeLearningResource(self):
        html_content = ""
        html_content +="<h1><u><b>"
        html_content += "Overall Leetcode Tag Type Appear in the Question Count"
        html_content +="</b></u></h1>\n"
        html_content +="<table>\n"
        html_content +="<tr>\n"
        html_content +="  <th>Tag</th>\n"
        html_content +="  <th>Count</th>\n"
        html_content +="</tr>\n"
        df = pd.read_csv("Question List Tag Count.csv")
        for index, row in df.iterrows():
            html_content += "<tr>\n" 
            tag_html ="  <td>" + str(row["Tag"]) + "</td>\n" 
            count_html ="  <td>" + str(row["Appearance"]) + "</td>\n" 
            html_content += tag_html
            html_content += count_html
            html_content += "</tr>\n"
        html_content +="</table>\n"

        html_content +="<h1><u><b>"
        html_content += "Top 100 Leetcode Question Tag Type Appear in the Question Count"
        html_content +="</b></u></h1>\n"
        html_content +="<table>\n"
        html_content +="<tr>\n"
        html_content +="  <th>Tag</th>\n"
        html_content +="  <th>Count</th>\n"
        html_content +="</tr>\n"
        df = pd.read_csv("Top 100 Question List Tag Count.csv")
        for index, row in df.iterrows():
            html_content += "<tr>\n" 
            tag_html ="  <td>" + str(row["Tag"]) + "</td>\n" 
            count_html ="  <td>" + str(row["Appearance"]) + "</td>\n" 
            html_content += tag_html
            html_content += count_html
            html_content += "</tr>\n"
        html_content +="</table>\n"

        for i in self.all_tags_list:
            path = "Leetcode Learning Resource/"+i+".html"
            if os.path.isfile(path) == False:
                print(d,"not found in",path)
            else:
                with open(path, "r", encoding="utf-8") as file:
                    title = i
                    html_content +="<h1><u><b>"
                    html_content += title
                    html_content +="</b></u></h1>"
                    html_content += file.read()
                    file.close()
        with open("leetcode learning resource.html", 'w', encoding='utf-8') as file:
            file.write(html_content)  
            file.close()
        output = pypandoc.convert_text(html_content, 'docx', format='html', outputfile='leetcode learning resource.docx')
        if output == "":
            print("Document output sucessfully.")
        else:
            print("Document output failed")

    def ConvertQuestionListToDataframeAndCountTag(self):
        tag_count_dict_list = {}
        for t in self.all_tags_list:
            tag_count_dict_list[t] = 0
        df = pd.read_csv("Question List.csv")
        #difficulty = pd.get_dummies(df['Difficulty']).astype(int)
        #df = df.drop(columns=['Difficulty'])
        #df = pd.concat([df,difficulty], axis= 1)
        tags_header_list = []
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                if tag not in tags_header_list:
                    tags_header_list.append(tag)
                if tag in tag_count_dict_list:
                    tag_count_dict_list[tag]+= 1
        for tag in tags_header_list:
            df[tag] = 0
        df2 = df
        for index, row in df2.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                df.loc[index, tag] = 1
        df = df.drop(columns=['Tags'])
        write_file_path = "Question List Dataframe" + ".csv"
        df.to_csv(write_file_path, encoding='utf-8', index=False)
        with open("Question List Tag Count" + ".csv", 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag", "Appearance"])
            for s in sorted(tag_count_dict_list, key=tag_count_dict_list.get, reverse=True):
                writer.writerow([s, tag_count_dict_list[s]])
            file.close()
            
    def MakeTop100QuestionTag(self):
        tag_count_dict_list = {}
        for t in self.all_tags_list:
            tag_count_dict_list[t] = 0
        df_a = pd.read_csv("Question List.csv")
        df = df_a.head(100)
        df.to_csv("Top 100 Question List.csv", encoding='utf-8', index=False)
        #difficulty = pd.get_dummies(df['Difficulty']).astype(int)
        #df = df.drop(columns=['Difficulty'])
        #df = pd.concat([df,difficulty], axis= 1)
        tags_header_list = []
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                if tag not in tags_header_list:
                    tags_header_list.append(tag)
                if tag in tag_count_dict_list:
                    tag_count_dict_list[tag]+= 1
        for tag in tags_header_list:
            df[tag] = 0
        df2 = df
        for index, row in df2.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                df.loc[index, tag] = 1
        df = df.drop(columns=['Tags'])
        write_file_path = "Top 100 Question List Dataframe" + ".csv"
        df.to_csv(write_file_path, encoding='utf-8', index=False)
        with open("Top 100 Question List Tag Count" + ".csv", 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag", "Appearance"])
            for s in sorted(tag_count_dict_list, key=tag_count_dict_list.get, reverse=True):
                writer.writerow([s, tag_count_dict_list[s]])
            file.close()


    def ConvertToDataframe(self):
        dest_folder = 'Companies Leetcode DataFrame'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
                       
        for c in self.companies_list:
            read_filename = c
            read_filename +='.csv'
            read_file_path = os.path.join('Companies Leetcode', read_filename)
            df = pd.read_csv(read_file_path)
            #difficulty = pd.get_dummies(df['Difficulty']).astype(int)
            #df = df.drop(columns=['Difficulty'])
            #df = pd.concat([df,difficulty], axis= 1)
            tags_header_list = []
            for index, row in df.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    if tag not in tags_header_list:
                        tags_header_list.append(tag)
            for tag in tags_header_list:
                df[tag] = 0
            df2 = df
            for index, row in df2.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    df.loc[index, tag] = 1
            df = df.drop(columns=['Tags','Link'])
            file_name = c + ".csv"
            write_file_path = os.path.join(dest_folder, file_name)
            df.to_csv(write_file_path, encoding='utf-8', index=False)
            print(c,"file successfully converted to data frame format.")

    def MakeDocsFromHtml(self):
        dir = 'Question'
        filenames = [f for f in listdir(dir) if isfile(join(dir, f))]
        for f in filenames:
            print(f)
            words = f.rsplit(".")
            extension = words[len(words)-1]
            if extension =="html":
                filename = f.replace(".html","")
                output = pypandoc.convert_file(dir + "/" + f, 'docx', outputfile= dir + "/" + filename +".docx")

 
    

In [6]:
test = LeetcodeDataProcessing()
test.GenerateLeetCodeLearningResource()

Document output sucessfully.


In [10]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

class LeetcodeAnalytics:
    def __init__(self):
        self.model_param={
            'LinearRegression': {
                'model': LinearRegression(),
                'param':{
                    'criterion': ['gini'],
                    'max_depth': range(2, 10),
                    'min_samples_split': range(2,100),
                }
            },
            'DecisionTreeRegressor': {
                'model': DecisionTreeRegressor(),
                'param': {
                    'criterion': ['squared_error','friedman_mse'],
                    'max_depth': range(2, 20),
                    'min_samples_split': range(2, 50),
                }
            },
            'RandomForestClassifier': {
                'model': RandomForestClassifier(),
                'param':{
                    'criterion':['gini'],
                    'max_depth': range(2, 4),
                    'n_estimators': [10, 50, 100, 130],
                }
            },
            'XGBClassifier': {
                'model': XGBClassifier(objective='binary:logistic'),
                'param':{
                    'learning_rate': [0.5, 0.1, 0.01, 0.001],
                    'max_depth': range(2, 20),
                    'n_estimators': range(10, 100,10),
                }
            }
        }
    def FindHardestTag(self):
       
        df = pd.read_csv("Question List.csv")
        tags_header_list = []
        top_dict = {}
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                if tag not in tags_header_list:
                    tags_header_list.append(tag)
        
        df = pd.read_csv("Question List Dataframe.csv")
        df["Acceptance"] = df["Acceptance"].str.replace('%', '').astype(float)
        df["Acceptance"] = 100 - df["Acceptance"]
        df["Acceptance"] = df["Acceptance"] / 100
        difficulty = pd.get_dummies(df['Difficulty']).astype(int)
        #df = pd.concat([df,difficulty], axis= 1)
        df = df.drop(columns=['No','Title','Link', 'Difficulty', 'Frequency'])
        y = df["Acceptance"]
        df = df.drop(labels='Acceptance', axis=1)
        df = df.astype('int64')
        x = df
       
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
        model = LinearRegression(n_jobs=-1)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        mae = mean_absolute_error(y_test, predictions)
        mape = mean_absolute_percentage_error(y_test, predictions)
        print(f'Mean Absolute Error: {mae}')
        print(f'Mean Absolute Percent Error: {mape}')

        hard_dict = {}
        new_df = x.head(1).copy()
        new_df.iloc[0] = 0
        new_df = new_df.astype('int64')
        for tag in tags_header_list:
            new_df.iloc[0] = 0
            new_df.at[0,tag]= 1
            predicted_score = model.predict(new_df)
            hard_dict[tag] = predicted_score


        #new_df.iloc[0] = 0
        #new_df = new_df.astype('int64')
        #for i in range(0,2361183241434822606847):
         #   binary_str = bin(i)
          #  binary_digits = binary_str[2:]
           # current_bit = len(binary_digits)
            #current_bit -= 1
            #c =0
            #while current_bit > -1:
             #   binary = int(binary_digits[current_bit])
              #  new_df.iloc[0, c] = binary
               # c+= 1
                #current_bit -= 1
           # new_df = new_df.astype('int64')
            #predicted_score = model.predict(new_df)
            #print(binary_digits, predicted_score)

    
        write_file_path = "Question List Hardest Tag" + ".csv"
        with open(write_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag", "Minus Acceptance rate"])
            for s in sorted(hard_dict, key=hard_dict.get, reverse=True):
                #print(s, hard_dict[s])
                writer.writerow([s, hard_dict[s]])
            file.close()
           
   
    def Analytics(self,name):
        dest_folder = 'Leetcode Analytics'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        read_filename = name
        read_filename +='.csv'
        read_file_path = os.path.join('Companies Leetcode', read_filename)
        df = pd.read_csv(read_file_path)
        tags_header_list = []
        top_dict = {}
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                if tag not in tags_header_list:
                    tags_header_list.append(tag)
                    top_dict[tag]=0
                top_dict[tag] += 1

        print(name)
        dest_folder = 'Top Tag'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        file_name = name + ".csv"
        write_file_path = os.path.join(dest_folder, file_name)
        with open(write_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag", "Appearance"])
            print("Top tag appear in the question.")
            for s in sorted(top_dict, key=top_dict.get, reverse=True):
                #print(s, top_dict[s])
                writer.writerow([s, top_dict[s]])
            file.close()
        top = sorted(top_dict, key=top_dict.get, reverse=True)[:5]
        print(top)
        read_file_path = os.path.join('Companies Leetcode DataFrame', read_filename)
        df = pd.read_csv(read_file_path)
        
        if df['No'].count() <= 3:
            print("\n")
            return  
        
        df["Acceptance"] = df["Acceptance"].str.replace('%', '').astype(float)
        df["Acceptance"] = df["Acceptance"] / 100
        difficulty = pd.get_dummies(df['Difficulty']).astype(int)
        #df = pd.concat([df,difficulty], axis= 1)
        df = df.drop(columns=['Title', 'Difficulty', 'Frequency'])

        #df.drop_duplicates(subset=['A', 'C'], keep=False)
        y = df["Acceptance"]
        x = df.drop(labels='Acceptance', axis=1)
      
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
        model = LinearRegression(n_jobs=-1)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        mae = mean_absolute_error(y_test, predictions)
        mape = mean_absolute_percentage_error(y_test, predictions)
        print(f'Mean Absolute Error: {mae}')
        print(f'Mean Absolute Percent Error: {mape}')
        column_names = df.columns
        hard_dict = {}

        for tag in tags_header_list:
            new_df = pd.DataFrame(columns=column_names)
            new_row = pd.Series(0, index=new_df.columns)
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
            new_df.at[0,tag]= 1
            #if 'Hard' in new_df.columns:
            #    new_df.at[0,'Hard']= 1
            #elif 'Medium' in new_df.columns:
            #    new_df.at[0,'Medium']= 1
           # else:
             #   new_df.at[0,'Easy']= 1
                   
            new_df = new_df.drop(labels='Acceptance', axis=1)
            first_row = new_df.iloc[[0]]
          
               
            predicted_score = model.predict(first_row)
            hard_dict[tag] = predicted_score

    

        dest_folder = 'Hardest Tag'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        file_name = name + ".csv"
        write_file_path = os.path.join(dest_folder, file_name)
        with open(write_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag", "Acceptance rate"])
            print("Hardest tag quesion.")
            for s in sorted(hard_dict, key=hard_dict.get, reverse=False):
                #print(s, hard_dict[s])
                writer.writerow([s, hard_dict[s]])
            file.close()
            hardest = sorted(hard_dict, key=hard_dict.get, reverse=False)[:5]
            print(hardest)
        print("\n")

    def AnalyticsAllLeetcode(self):
        cf = open("companies.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            companies_list.append(c)
        cf.close

        for c in companies_list:
            self.Analytics(c)
        print("Done")

    
    

In [11]:
test =LeetcodeAnalytics()
test.FindHardestTag()


Mean Absolute Error: 0.12057996962313869
Mean Absolute Percent Error: 0.38229830825311323
